<a href="https://colab.research.google.com/github/serasset/llod-cloud-evaluator/blob/main/BATS_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transforming BATS to RDF

Example of BATS structure: 
```
acrobat	troupe
antelope	herd
bee	swarm/hive/colony/drift/cast/nest/rabble/stand
bird	flock
book	library/shelf/collection/accounting/accounting_system/method_of_accounting
````

Modeling in RDF, using Ontolex:

```
PREFIX...

:acrobat a ontolex:LexicalEntry;
  rdfs:label "acrobat"@en.

:troupe a ontolex:LexicalEntry;
  rdfs:label "troupe"@en.

:meroMemember_01 a rdf:Relation;
  rdf:source :acrobat ;
  rdf:target :troupe ;
  rdf:property bats:meronym_member ;
  bats:rank "1"@int .

```


In [ ]:
from rdflib import URIRef, BNode, Literal, Namespace

with open(...) as f:
    
    for line in f:
      
      terms = line.split()
      entry = terms[0]
      associated_terms = terms.split('/')


      # TODO: SPECIFY PREFIX - bats url
      prefix_namespace = Namespace("...")

      # TODO: specify ontolex
      onto_namespace = Namespace("...")
      onto_namespace.LexicalEntry

      subj = URIRef(".../{entry}")

      g = Graph()
      g.bind(prefix_namespace)

      g.add((subj, RDF.type, onto_namespace.LexicalEntry))
      g.add()






      g.serialize()






      
